In [1]:
%load_ext autoreload
%autoreload 2

# TruLens for LLMs: Quickstart

In this quickstart you will create a simple LLM Chain and learn how to log it and get feedback on an LLM response.

Note: If you haven't already, make sure to set up your local .env file with your OpenAI and Huggingface Keys. Your .env file should be in the same directory that you run this notebook. If you need help, take a look at the [.env.example](https://github.com/truera/trulens_private/blob/e8b11c4689e644687d2eafe09d90d8d7774b581c/trulens_eval/trulens_eval/.env.example#L4)

## Import from LangChain and TruLens

In [2]:
from IPython.display import JSON

# imports from langchain to build app
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

# imports from trulens to log and get feedback on chain
from trulens_eval import TruChain, Feedback, OpenAI, Huggingface, db, tru
# from trulens_eval import tru_chain
# from trulens_eval.tru_chain import TruChain
# from trulens_eval.tru_feedback import Feedback, Huggingface, OpenAI
from trulens_eval.keys import *

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


## Create Simple LLM Application

This example uses a LangChain framework and OpenAI LLM

In [3]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)

chain = LLMChain(llm=chat, prompt=chat_prompt_template, verbose=True)

## Send your first request to your new app, asking how to adopt a dog in spanish

In [4]:
prompt_input = 'Como adopto un perro?'
gpt3_response = chain(prompt_input)

display(gpt3_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: Como adopto un perro?

> Finished chain.


{'prompt': 'Como adopto un perro?',
 'text': 'Adoptar un perro es una gran responsabilidad y requiere una cuidadosa consideración antes de tomar cualquier decisión. Hay varias opciones para adoptar un perro, incluyendo refugios de animales, organizaciones de rescate de perros y criadores de perros responsables. Antes de adoptar, es importante investigar y elegir una fuente confiable para evitar problemas de comportamiento o de salud en el futuro.\n\nUna vez que hayas decidido adoptar, puedes visitar los refugios de animales o contactar con organizaciones de rescate para conocer a los perros disponibles para adopción. Los trabajadores del refugio o los voluntarios de la organización pueden ayudarte a encontrar un perro que se adapte a tu estilo de vida y personalidad. También te pueden proporcionar información sobre los requisitos de adopción, como la necesidad de un hogar seguro y adecuado para el perro.\n\nEs importante tener en cuenta que la adopción de un perro involucra una inversi

# Instrument chain for logging with TruLens

In [5]:
truchain: TruChain = TruChain(chain, chain_id='Chain1_ChatApplication')

In [6]:
# Instrumented chain can operate like the original:

gpt3_response = truchain(prompt_input)

display(gpt3_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: Como adopto un perro?

> Finished chain.
*** Creating new TP singleton instance for name = None ***


{'prompt': 'Como adopto un perro?',
 'text': '¡Adoptar un perro es una excelente manera de agregar un nuevo miembro a tu familia y ayudar a un animal que necesita un hogar amoroso! \n\nPara adoptar un perro, hay varios pasos que puedes tomar. Primero, investiga refugios y organizaciones de rescate en tu área. Por lo general, tendrán una lista de perros disponibles para la adopción en su sitio web. \n\nVisita el refugio o la organización de rescate y pasa tiempo interactuando con los perros que parezcan ser buenos candidatos para tu hogar. Pregúntale al personal del refugio sobre el temperamento y las necesidades de cada perro para asegurarte de que sea compatible con tu estilo de vida.\n\nUna vez que hayas encontrado el perro adecuado, completarás una solicitud de adopción y se realizará una verificación de antecedentes para asegurarse de que eres un buen candidato para ser dueño del perro. Si todo va bien, podrás llevar al perro a casa y comenzar a disfrutar de la vida juntos.\n\nRecu

In [7]:
# But can also produce a log or "record" of the execution of the chain:

gpt3_response, record = truchain.call_with_record(prompt_input)

JSON(record)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: Como adopto un perro?

> Finished chain.


<IPython.core.display.JSON object>

In [8]:
# We can log the records but first we need to log the chain itself:

tru.add_chain(chain_json=truchain.json)

In [9]:
# Now the record:

record_id = tru.add_record(
    prompt=prompt_input, # prompt input
    response=gpt3_response['text'], # LLM response
    record_json=record # record is returned by the TruChain wrapper
)

In [10]:
# Initialize Huggingface-based feedback function collection class:
hugs = Huggingface()

# Define a language match feedback function using HuggingFace.
f_lang_match = Feedback(hugs.language_match).on(text1="prompt", text2="response")

# from trulens_eval import tru_db
# db = tru_db.LocalSQLite("someotherfile.sql.db")

# wrap with truchain
# tc = tru_chain.TruChain(
#     chain, db=db, feedbacks=[f_lang_match]
#)

# This might take a moment if the public api needs to load the language model used in the feedback function:
feedback_result = f_lang_match.run_on_record(chain_json=truchain.json, record_json=record)

*** Creating new Endpoint singleton instance for name = huggingface ***
*** Creating huggingface endpoint ***


huggingface api: 0requests [00:00, ?requests/s]

In [11]:
feedback_result

{'_success': True, 'language_match': 0.00496305342312553}

In [ ]:
JSON(truchain.json)

## Log data and feedback

In [ ]:
record_id = tru.add_data(
        chain_id='Chain1_ChatApplication', #name your chain
        prompt=prompt_input, # prompt input
        response=gpt3_response['text'], # LLM response
        record=record, # record is returned by the TruChain wrapper
        tags='dev' #add a tag
    )


# initialize Huggingface class for feedback function generation
hugs = Huggingface()

# Generate a language match feedback function using HuggingFace
f_lang_match = Feedback(hugs.language_match).on(text1="prompt", text2="response")

# Run feedack functions
feedback = tru.run_feedback_functions(
        chain=chain, # the unwrapped chain
        record=record, # record is returned by the TruChain wrapper
        feedback_functions=[f_lang_match] # a list of feedback functions to apply
    )

tru.add_feedback(record_id, feedback) # log the feedback by providing the record id

## Run the TruLens dashboard to explore the quality of your LLM chain

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore